In [ ]:
import os
from glob import glob
from tqdm import tqdm
import cv2
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torchvision.utils import save_image
import itertools
import random
from PIL import Image
import torch.utils.data as data
from glob import glob
from tqdm import tqdm
from torchvision.utils import save_image
import random
import matplotlib.pyplot as plt 
topilimage =transforms.ToPILImage()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, features):
        super(ResidualBlock, self).__init__()
        self.block = nn.Sequential(
            nn.Conv2d(features, features, kernel_size=3, padding=1),
            nn.InstanceNorm2d(features),
            nn.ReLU(inplace=True),
            nn.Conv2d(features, features, kernel_size=3, padding=1),
            nn.InstanceNorm2d(features),
            nn.Dropout(0.5)  # Dropout 추가 (드롭아웃 확률 0.5)
        )

    def forward(self, x):
        return x + self.block(x)

# Generator Model
class Generator(nn.Module):
    def __init__(self, input_channels, output_channels, n_residual_blocks=9):
        super(Generator, self).__init__()
        # 초기 컨볼루션 블록
        model = [
            nn.Conv2d(input_channels, 64, kernel_size=7, padding=3),
            nn.InstanceNorm2d(64),
            nn.ReLU(inplace=True)
        ]

        # 다운샘플링
        in_features = 64
        out_features = in_features * 2
        for _ in range(4):  # 기존 2에서 4로 변경
            model += [
                nn.Conv2d(in_features, out_features, kernel_size=3, stride=2, padding=1),
                nn.InstanceNorm2d(out_features),
                nn.ReLU(inplace=True)
            ]
            in_features = out_features
            out_features = in_features * 2

        # 잔차 블록
        for _ in range(n_residual_blocks):
            model += [ResidualBlock(in_features)]

        # 업샘플링
        out_features = in_features // 2
        for _ in range(4):  # 기존 2에서 4로 변경
            model += [
                nn.ConvTranspose2d(in_features, out_features, kernel_size=3, stride=2, padding=1, output_padding=1),
                nn.InstanceNorm2d(out_features),
                nn.ReLU(inplace=True)
            ]
            in_features = out_features
            out_features = in_features // 2

        # 출력 레이어
        model += [nn.Conv2d(64, output_channels, kernel_size=7, padding=3), nn.Tanh()]
        self.model = nn.Sequential(*model)

    def forward(self, x):
        return self.model(x)

# Discriminator Model
class Discriminator(nn.Module):
    def __init__(self, input_channels):
        super(Discriminator, self).__init__()
        
        def discriminator_block(in_filters, out_filters, normalization=True):
            layers = [nn.Conv2d(in_filters, out_filters, kernel_size=4, stride=2, padding=1)]
            if normalization:
                layers.append(nn.InstanceNorm2d(out_filters))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *discriminator_block(input_channels, 64, normalization=False),
            *discriminator_block(64, 128),
            *discriminator_block(128, 256),
            *discriminator_block(256, 512),
            nn.ZeroPad2d((1, 0, 1, 0)),
            nn.Conv2d(512, 1, kernel_size=4, padding=1)
        )

    def forward(self, img):
        return self.model(img)
    
# Initialize and load model
F = Generator(3, 3).to(device)
model_path = '../../model/HE_IHC_translation/internal_ss/PD-L1/F_99.pth'
F.load_state_dict(torch.load(model_path, map_location=device))
F.eval()
transform = transforms.Compose([
    transforms.Resize(size=512),
    transforms.ToTensor()
])
print(f"✓ Loaded model: {model_path}")
print(f"✓ Generator F (IHC → HE) ready for testing")

In [ ]:
pdl1_guide_src_list=glob('../../data/IHC_HE_Pair_Data_GA_SS/patches/pdl1_mpp1/*.png')
pdl1_guide_path='../../data/IHC_HE_Pair_Data_GA_SS/patches/pdl1_guide/'
he_guide_path='../../data/IHC_HE_Pair_Data_GA_SS/patches/he_guide/'

create_dir(pdl1_guide_path)
create_dir(he_guide_path)

In [ ]:
for i in tqdm(range(len(pdl1_guide_src_list))):

    filename=os.path.basename(pdl1_guide_src_list[i])
    pdl1_img=Image.open(pdl1_guide_src_list[i]).convert('RGB')
    pdl1_tensor=transform(pdl1_img).unsqueeze(0).to(device)*2.-1.
    with torch.no_grad():
        he_tensor=F(pdl1_tensor)
    he_img=he_tensor.squeeze(0).cpu()*0.5+0.5
    pdl1_img=pdl1_tensor.squeeze(0).cpu()*0.5+0.5
    he_np_img = np.array(he_img.permute(1, 2, 0)*255).astype(np.uint8)
    he_with_rect = he_np_img.copy()  # 이미지 복사
    cv2.rectangle(he_with_rect, (128,128), (384,384), (0,255,0), 2)  
    pdl1_np_img=np.array(pdl1_img.permute(1, 2, 0)*255).astype(np.uint8)
    pdl1_with_rect = pdl1_np_img.copy()  # 이미지 복사
    cv2.rectangle(pdl1_with_rect, (128,128), (384,384), (0,255,0), 2)  
    he_with_rect_img=Image.fromarray(he_with_rect)
    pdl1_with_rect_img=Image.fromarray(pdl1_with_rect)
    pdl1_with_rect_img.save(os.path.join(pdl1_guide_path,filename.split('.')[0]+'_pdl1_guide.tiff'))
    he_with_rect_img.save(os.path.join(he_guide_path,filename.split('.')[0]+'_he_guide.tiff'))



In [ ]:
filename.split('.')[0]